In [53]:
import pandas as pd

def translate_team(team_name):
    # Check if translation exists, otherwise return the original name
    return translation_dict.get(team_name, f"Unknown ({team_name})")

Head_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\prediction_data/'
# Step 1: Load the prediction CSV
prediction_df = pd.read_csv(Head_path+'prediction.csv')

# Step 2: Load the translation CSV (assuming it has 'Chinese' and 'English' columns)
translation_df = pd.read_csv(Head_path+'translation.csv', encoding='utf-8')

# Step 3: Create a dictionary for mapping Chinese team names to English
translation_dict = pd.Series(translation_df['English'].values, index=translation_df['Chinese'].values).to_dict()

# Step 3: Create a dictionary for mapping Chinese aHh to Math
translation_dict_math = pd.Series(translation_df['Math_Hh'].values, index=translation_df['Chinese_Hh'].values).to_dict()

# Step 4: Replace Chinese team names with English in HomeTeam and AwayTeam columns
prediction_df['HomeTeam'] = prediction_df['HomeTeam'].map(translation_dict).fillna(prediction_df['HomeTeam'])
prediction_df['AwayTeam'] = prediction_df['AwayTeam'].map(translation_dict).fillna(prediction_df['AwayTeam'])
prediction_df['AHh'] = prediction_df['AHh'].map(translation_dict_math).fillna(prediction_df['AHh'])
# prediction_df['HomeTeam'] = prediction_df['HomeTeam'].apply(translate_team)
# prediction_df['AwayTeam'] = prediction_df['AwayTeam'].apply(translate_team)

# Step 5: Save the updated dataframe (optional)
prediction_df.to_csv(Head_path+'updated_prediction.csv', index=False, encoding='utf-8-sig')
print(prediction_df)

    Unnamed: 0  Div       Date     HomeTeam       AwayTeam  B365H  B365D  \
0            0   I1  2025/3/16      Venezia         Napoli   7.50   4.10   
1            1  SP1  2025/3/16      Leganes          Betis   3.80   3.30   
2            2   E0  2025/3/16      Arsenal        Chelsea   1.75   3.50   
3            3   E0  2025/3/16       Fulham      Tottenham   1.90   3.70   
4            4   I1  2025/3/16      Bologna          Lazio   2.25   3.00   
5            5   D1  2025/3/16       Bochum  Ein Frankfurt   2.75   3.70   
6            6   I1  2025/3/16         Roma       Cagliari   1.48   4.33   
7            7  SP1  2025/3/16      Sevilla     Ath Bilbao   2.90   3.10   
8            8   F1  2025/3/16  Montpellier     St Etienne   2.20   3.50   
9            9   F1  2025/3/16        Brest          Reims   1.72   3.80   
10          10   F1  2025/3/16   Strasbourg       Toulouse   2.40   3.20   
11          11   E0  2025/3/16    Liverpool      Newcastle   4.33   3.90   
12          

In [54]:
# import os
# 
# # 文件夹路径
# folder_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\elo/'
# 
# # 获取文件夹下所有文件的名称
# file_names = []
# for root, dirs, files in os.walk(folder_path):
#     for file in files:
#         # 分离文件名和扩展名，获取文件名部分
#         file_name_without_extension = os.path.splitext(file)[0]
#         file_names.append(file_name_without_extension)
# 
# # 将文件名称写入到 txt 文件，按行存储
# with open(folder_path + 'file_names.txt', 'w', encoding='utf-8') as f:
#     for file_name in file_names:
#         f.write(file_name + '\n')
# 
# print("文件名称已成功保存到 file_names.txt")

In [55]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import joblib
from joblib import load, dump
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [56]:
df = pd.read_csv('prediction_data/updated_prediction.csv', low_memory = False)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,balance_val_model,model_v4,info,FTR_real,predict1.1,predict2.1,predict3.1,model_FTR,info_FTR,match_div_info
0,0,I1,2025/3/16,Venezia,Napoli,7.50,4.10,1.48,1.00,2.05,...,1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,SP1,2025/3/16,Leganes,Betis,3.80,3.30,2.05,0.50,1.84,...,1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,E0,2025/3/16,Arsenal,Chelsea,1.75,3.50,5.00,-0.75,2.04,...,-1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,E0,2025/3/16,Fulham,Tottenham,1.90,3.70,3.80,-0.50,1.91,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,I1,2025/3/16,Bologna,Lazio,2.25,3.00,3.50,-0.25,1.97,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,D1,2025/3/16,Bochum,Ein Frankfurt,2.75,3.70,2.40,0.25,1.82,...,1,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,I1,2025/3/16,Roma,Cagliari,1.48,4.33,6.50,-1.00,1.85,...,-1,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,SP1,2025/3/16,Sevilla,Ath Bilbao,2.90,3.10,2.55,0.00,2.09,...,0,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,F1,2025/3/16,Montpellier,St Etienne,2.20,3.50,3.20,-0.25,1.95,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,F1,2025/3/16,Brest,Reims,1.72,3.80,4.75,-0.75,1.95,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
conditions = [
    df['AHh'] >= 2.5,         # AHh >= 2.75
    (df['AHh'] >= 1.5) & (df['AHh'] <= 2.25),  # 1.75 < AHh <= 2.75
    (df['AHh'] >= 0.25) & (df['AHh'] <= 1.25),  # 0.25 < AHh <= 1.75
    df['AHh'] == 0,            # AHh == 0
    (df['AHh'] >= -1.25) & (df['AHh'] <= -0.25),  # -1.75 < AHh <= -0.25
    (df['AHh'] >= -2.25) & (df['AHh'] <= -1.5),  # -2.75 < AHh <= -1.75
    df['AHh'] <= -2.5
]
labels = [3, 2, 1, 0, -1, -2, -3]

df['balance_val'] = np.select(conditions, labels)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,balance_val_model,model_v4,info,FTR_real,predict1.1,predict2.1,predict3.1,model_FTR,info_FTR,match_div_info
0,0,I1,2025/3/16,Venezia,Napoli,7.50,4.10,1.48,1.00,2.05,...,1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,SP1,2025/3/16,Leganes,Betis,3.80,3.30,2.05,0.50,1.84,...,1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,E0,2025/3/16,Arsenal,Chelsea,1.75,3.50,5.00,-0.75,2.04,...,-1,[2],反向 特殊处理2:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,E0,2025/3/16,Fulham,Tottenham,1.90,3.70,3.80,-0.50,1.91,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,I1,2025/3/16,Bologna,Lazio,2.25,3.00,3.50,-0.25,1.97,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,D1,2025/3/16,Bochum,Ein Frankfurt,2.75,3.70,2.40,0.25,1.82,...,1,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,I1,2025/3/16,Roma,Cagliari,1.48,4.33,6.50,-1.00,1.85,...,-1,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,SP1,2025/3/16,Sevilla,Ath Bilbao,2.90,3.10,2.55,0.00,2.09,...,0,[2],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,F1,2025/3/16,Montpellier,St Etienne,2.20,3.50,3.20,-0.25,1.95,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,F1,2025/3/16,Brest,Reims,1.72,3.80,4.75,-0.75,1.95,...,-1,[0],默认单一预测:,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A","AHh", "B365AHH", "B365AHA", "balance_val"
                    ]
df = df[features_to_keep]

In [59]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    # if date > '2000-05-01':
    #     aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("prediction_data/updated_prediction_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,I1,2025/3/16,Venezia,Napoli,7.50,4.10,1.48,1.00,2.05,1.85,1,1555.340942,1838.037354
1,SP1,2025/3/16,Leganes,Betis,3.80,3.30,2.05,0.50,1.84,2.06,1,1584.337036,1713.384399
2,E0,2025/3/16,Arsenal,Chelsea,1.75,3.50,5.00,-0.75,2.04,1.86,-1,1988.350342,1846.273926
3,E0,2025/3/16,Fulham,Tottenham,1.90,3.70,3.80,-0.50,1.91,1.99,-1,1766.227051,1792.356689
4,I1,2025/3/16,Bologna,Lazio,2.25,3.00,3.50,-0.25,1.97,1.93,-1,1772.388428,1793.554688
5,D1,2025/3/16,Bochum,Ein Frankfurt,2.75,3.70,2.40,0.25,1.82,2.08,1,1556.519409,1714.377319
6,I1,2025/3/16,Roma,Cagliari,1.48,4.33,6.50,-1.00,1.85,2.05,-1,1798.377686,1603.342651
7,SP1,2025/3/16,Sevilla,Ath Bilbao,2.90,3.10,2.55,0.00,2.09,1.81,0,1688.584717,1790.508179
8,F1,2025/3/16,Montpellier,St Etienne,2.20,3.50,3.20,-0.25,1.95,1.95,-1,1537.360840,1495.714844
9,F1,2025/3/16,Brest,Reims,1.72,3.80,4.75,-0.75,1.95,1.95,-1,1678.191284,1565.918945


In [60]:
# 提前9个月，用来划分赛季
# 将字符串转换为日期类型，指定日期格式
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df['Date'] = df['Date'] - pd.DateOffset(months=6)
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,I1,2024-09-16,Venezia,Napoli,7.50,4.10,1.48,1.00,2.05,1.85,1,1555.340942,1838.037354
1,SP1,2024-09-16,Leganes,Betis,3.80,3.30,2.05,0.50,1.84,2.06,1,1584.337036,1713.384399
2,E0,2024-09-16,Arsenal,Chelsea,1.75,3.50,5.00,-0.75,2.04,1.86,-1,1988.350342,1846.273926
3,E0,2024-09-16,Fulham,Tottenham,1.90,3.70,3.80,-0.50,1.91,1.99,-1,1766.227051,1792.356689
4,I1,2024-09-16,Bologna,Lazio,2.25,3.00,3.50,-0.25,1.97,1.93,-1,1772.388428,1793.554688
5,D1,2024-09-16,Bochum,Ein Frankfurt,2.75,3.70,2.40,0.25,1.82,2.08,1,1556.519409,1714.377319
6,I1,2024-09-16,Roma,Cagliari,1.48,4.33,6.50,-1.00,1.85,2.05,-1,1798.377686,1603.342651
7,SP1,2024-09-16,Sevilla,Ath Bilbao,2.90,3.10,2.55,0.00,2.09,1.81,0,1688.584717,1790.508179
8,F1,2024-09-16,Montpellier,St Etienne,2.20,3.50,3.20,-0.25,1.95,1.95,-1,1537.360840,1495.714844
9,F1,2024-09-16,Brest,Reims,1.72,3.80,4.75,-0.75,1.95,1.95,-1,1678.191284,1565.918945


In [61]:
df_noNa = df.dropna()
df_noNa.reset_index(inplace=True, drop=True)
df_noNa

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,I1,2024-09-16,Venezia,Napoli,7.50,4.10,1.48,1.00,2.05,1.85,1,1555.340942,1838.037354
1,SP1,2024-09-16,Leganes,Betis,3.80,3.30,2.05,0.50,1.84,2.06,1,1584.337036,1713.384399
2,E0,2024-09-16,Arsenal,Chelsea,1.75,3.50,5.00,-0.75,2.04,1.86,-1,1988.350342,1846.273926
3,E0,2024-09-16,Fulham,Tottenham,1.90,3.70,3.80,-0.50,1.91,1.99,-1,1766.227051,1792.356689
4,I1,2024-09-16,Bologna,Lazio,2.25,3.00,3.50,-0.25,1.97,1.93,-1,1772.388428,1793.554688
5,D1,2024-09-16,Bochum,Ein Frankfurt,2.75,3.70,2.40,0.25,1.82,2.08,1,1556.519409,1714.377319
6,I1,2024-09-16,Roma,Cagliari,1.48,4.33,6.50,-1.00,1.85,2.05,-1,1798.377686,1603.342651
7,SP1,2024-09-16,Sevilla,Ath Bilbao,2.90,3.10,2.55,0.00,2.09,1.81,0,1688.584717,1790.508179
8,F1,2024-09-16,Montpellier,St Etienne,2.20,3.50,3.20,-0.25,1.95,1.95,-1,1537.360840,1495.714844
9,F1,2024-09-16,Brest,Reims,1.72,3.80,4.75,-0.75,1.95,1.95,-1,1678.191284,1565.918945


In [62]:
# df_noNa_new = df_noNa.iloc[:3]
df_noNa_new = df_noNa

In [63]:
# 与历史数据合并，计算后续特征
history_df = pd.read_csv(Head_path+'history_data_FTR.csv')
history_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003-02-02,Auxerre,Nice,1,2,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003-02-02,Guingamp,Marseille,0,1,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003-02-02,Hamburg,Hannover,0,3,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003-02-02,Hertha,Werder Bremen,0,3,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003-02-02,Lens,Le Mans,0,0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37620,E0,2024-09-09,Tottenham,Bournemouth,2,2,D,2.700,3.900,2.350,2.62,3.8,2.4,0.00,2.080,1.820,1793.685059,1797.491943,1,0
37621,I1,2024-09-09,Verona,Bologna,1,2,A,5.250,3.300,1.750,4.80,3.3,1.8,0.75,1.850,2.050,1602.425171,1767.334839,1,1
37622,SP1,2024-09-10,Espanol,Girona,1,1,D,3.100,3.200,2.400,3.10,3.2,2.4,0.25,1.810,2.090,1616.476318,1687.630859,2,1
37623,I1,2024-09-10,Lazio,Udinese,1,1,D,1.730,3.500,5.250,1.73,3.6,4.8,-0.75,2.000,1.900,1798.021851,1673.347168,0,-1


In [64]:
# 合并两个 DataFrame，重置索引
df_noNa_comb = pd.concat([history_df, df_noNa_new], ignore_index=True)
df_noNa_comb

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003-02-02,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,1.850,1702.604858,1611.196045,0.0,-1
1,F1,2003-02-02,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,1.975,1685.016113,1665.625732,0.0,0
2,D1,2003-02-02,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,2.100,1718.566284,1649.805298,0.0,-1
3,D1,2003-02-02,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,1.875,1719.916748,1692.120972,0.0,-1
4,F1,2003-02-02,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,2.000,1697.354004,1539.958130,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,D1,2024-09-16 00:00:00,Stuttgart,Leverkusen,NaN,NaN,NaN,3.250,3.300,2.200,NaN,NaN,NaN,0.25,1.950,1.950,1716.876831,1878.866089,NaN,1
37642,E0,2024-09-16 00:00:00,Leicester,Man United,NaN,NaN,NaN,4.100,3.600,1.900,NaN,NaN,NaN,0.50,1.980,1.920,1594.307007,1767.211182,NaN,1
37643,I1,2024-09-16 00:00:00,Atalanta,Inter,NaN,NaN,NaN,2.750,3.250,2.620,NaN,NaN,NaN,0.00,1.990,1.910,1870.710083,1952.404785,NaN,0
37644,F1,2024-09-16 00:00:00,Paris SG,Marseille,NaN,NaN,NaN,1.420,5.000,6.500,NaN,NaN,NaN,-1.25,1.910,1.990,1943.813232,1739.462769,NaN,-1


In [65]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [66]:
# # 加载编码器
div_encoder = joblib.load('div_encoder.pkl')
home_encoder = joblib.load('home_encoder.pkl')
away_encoder = joblib.load('away_encoder.pkl')
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder
# 
# div_encoder = OneHotEncoder()
# home_encoder = OneHotEncoder()
# away_encoder = OneHotEncoder()
# 你可以使用加载的编码器进行转换
onehot_div = div_encoder.transform(df_noNa_comb.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.transform(df_noNa_comb.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.transform(df_noNa_comb.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

df_noNa_div = pd.concat([df_noNa_comb, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
df_noNa_div.drop(columns = ['Div'], inplace = True)
df_noNa_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003-02-02,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,...,0,0,0,0,0,0,0,0,0,0
1,2003-02-02,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,...,0,0,0,0,0,0,0,0,0,0
2,2003-02-02,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,...,0,0,0,0,0,0,0,0,0,0
3,2003-02-02,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,...,0,0,1,0,0,0,0,0,0,0
4,2003-02-02,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,2024-09-16 00:00:00,Stuttgart,Leverkusen,NaN,NaN,NaN,3.250,3.300,2.200,NaN,...,0,0,0,0,0,0,0,0,0,0
37642,2024-09-16 00:00:00,Leicester,Man United,NaN,NaN,NaN,4.100,3.600,1.900,NaN,...,0,0,0,0,0,0,0,0,0,0
37643,2024-09-16 00:00:00,Atalanta,Inter,NaN,NaN,NaN,2.750,3.250,2.620,NaN,...,0,0,0,0,0,0,0,0,0,0
37644,2024-09-16 00:00:00,Paris SG,Marseille,NaN,NaN,NaN,1.420,5.000,6.500,NaN,...,0,0,0,0,0,0,0,0,0,0


In [67]:
# 处理未进行比赛的easy_label
df_noNa_div['easy_label'] = df_noNa_div['easy_label'].fillna(1)  # 或其他有意义的值
df_noNa_div['FTR'] = df_noNa_div['FTR'].fillna('D')  # 或其他有意义的值

In [68]:
target_encoder = joblib.load('target_encoder_FTR.pkl')
# df_noNa_div['Result'] = target_encoder.transform(df_noNa_div.easy_label) 
df_noNa_div['Result'] = target_encoder.transform(df_noNa_div.FTR) 

In [69]:
df_noNa_div['Year'] = pd.DatetimeIndex(df_noNa_div.Date).year

df_noNa_div['Month'] = pd.DatetimeIndex(df_noNa_div.Date).month
df_noNa_div['Sin_Month'] = np.sin(2*np.pi*df_noNa_div.Month/12)
df_noNa_div['Cos_Month'] = np.cos(2*np.pi*df_noNa_div.Month/12)

df_noNa_div['DayofYear'] = pd.DatetimeIndex(df_noNa_div.Date).dayofyear
df_noNa_div['Sin_Day'] = np.sin(2*np.pi*df_noNa_div.DayofYear/365)
df_noNa_div['Cos_Day'] = np.cos(2*np.pi*df_noNa_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
# df_noNa_div_date = df_noNa_div.drop(columns = ['Date','Month'])
df_noNa_div_date = df_noNa_div.drop(columns = ['Date'])
df_noNa_div_date

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 205,AwayTeam 206,Result,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,1,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,Stuttgart,Leverkusen,NaN,NaN,D,3.250,3.300,2.200,NaN,NaN,...,0,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491
37642,Leicester,Man United,NaN,NaN,D,4.100,3.600,1.900,NaN,NaN,...,0,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491
37643,Atalanta,Inter,NaN,NaN,D,2.750,3.250,2.620,NaN,NaN,...,0,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491
37644,Paris SG,Marseille,NaN,NaN,D,1.420,5.000,6.500,NaN,NaN,...,0,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491


In [70]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.Result.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    df.drop(columns = ['FTR'], inplace = True)
    return df
df_noNa_div_date_lastMatch = compute_last_matches(df_noNa_div_date)
df_noNa_div_date_lastMatch

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,AwayTeam 206,Result,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,...,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,...,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,...,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,...,0,0,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,...,0,1,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,Stuttgart,Leverkusen,NaN,NaN,3.250,3.300,2.200,NaN,NaN,NaN,...,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,1
37642,Leicester,Man United,NaN,NaN,4.100,3.600,1.900,NaN,NaN,NaN,...,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,0
37643,Atalanta,Inter,NaN,NaN,2.750,3.250,2.620,NaN,NaN,NaN,...,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,0
37644,Paris SG,Marseille,NaN,NaN,1.420,5.000,6.500,NaN,NaN,NaN,...,0,1,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,2


In [71]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        # if year > 2024:
        #     # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
        #     year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
        #     year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            home_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(home_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['HomeWin'] == 1:  # 如果上一场比赛主队赢
                    home_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    home_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['HomeWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
            # team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            # team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            # year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            away_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(away_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['AwayWin'] == 1:  # 如果上一场比赛主队赢
                    away_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    away_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['AwayWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
            # team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            # team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            # year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])
df_noNa_div_date_lastMatch_streak = compute_winstreak(df_noNa_div_date_lastMatch)
df_noNa_div_date_lastMatch_streak

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,...,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3,0,0
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,...,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3,0,0
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,...,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,...,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3,0,0
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,...,2003,2,0.866025,5.000000e-01,33,0.538005,0.842942,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,Stuttgart,Leverkusen,NaN,NaN,3.250,3.300,2.200,NaN,NaN,NaN,...,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,1,0,2
37642,Leicester,Man United,NaN,NaN,4.100,3.600,1.900,NaN,NaN,NaN,...,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,0,0,0
37643,Atalanta,Inter,NaN,NaN,2.750,3.250,2.620,NaN,NaN,NaN,...,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,0,0,0
37644,Paris SG,Marseille,NaN,NaN,1.420,5.000,6.500,NaN,NaN,NaN,...,2024,9,-1.000000,-1.836970e-16,260,-0.972118,-0.234491,2,2,0


In [72]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        # if year > 2024:
        #     # 将 AwayWin = 3 当作 0 来处理，fangzhi计算连胜记录
        #     year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
        #     year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

             # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['HomeWinsToDate'] = team_df.HomeWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['HomeWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
            # team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            # year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['AwayWinsToDate'] = team_df.AwayWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['AwayWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            # team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            # year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])
df_noNa_div_date_lastMatch_streak_windate = compute_winstodate(df_noNa_div_date_lastMatch_streak)
df_noNa_div_date_lastMatch_streak_windate.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)
df_noNa_div_date_lastMatch_streak_windate

,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
1,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
2,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
3,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
4,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,NaN,NaN,3.250,3.300,2.200,NaN,NaN,NaN,0.25,1.950,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,1,0,2,6.0,7.0
37642,NaN,NaN,4.100,3.600,1.900,NaN,NaN,NaN,0.50,1.980,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,2.0,3.0
37643,NaN,NaN,2.750,3.250,2.620,NaN,NaN,NaN,0.00,1.990,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,7.0,8.0
37644,NaN,NaN,1.420,5.000,6.500,NaN,NaN,NaN,-1.25,1.910,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,2,2,0,10.0,9.0


In [73]:
asia_feats = ['AHh', 'B365AHH', 'B365AHA']
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [74]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]
df_odd = compute_meanodds(df_noNa_div_date_lastMatch_streak_windate, betting_feats)
df_odd.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG'], inplace = True)
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,-1,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,-1,1,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,-1,1,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,-1,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,3.250,3.300,2.200,0.25,1.950,1.950,1716.876831,1878.866089,1,1,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,1,0,2,6.0,7.0
37642,4.100,3.600,1.900,0.50,1.980,1.920,1594.307007,1767.211182,1,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,2.0,3.0
37643,2.750,3.250,2.620,0.00,1.990,1.910,1870.710083,1952.404785,0,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,7.0,8.0
37644,1.420,5.000,6.500,-1.25,1.910,1.990,1943.813232,1739.462769,-1,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,2,2,0,10.0,9.0


In [75]:
from sklearn.preprocessing import MinMaxScaler
# 加载保存的 MinMaxScaler
# minmax_scaler = joblib.load('minmax_scaler_2025_02_14_17_50_38.pkl')
minmax_scaler = joblib.load('minmax_scaler_2025_03_11_09_00_37.pkl')
# minmax_scaler = MinMaxScaler()
df_odd.loc[:, ['Year']] = minmax_scaler.transform(df_odd.loc[:, ['Year']])

from sklearn.preprocessing import StandardScaler
# std_scaler = joblib.load('std_scaler_2025_02_14_17_50_38.pkl')
std_scaler = joblib.load('std_scaler_2025_03_11_09_00_37.pkl')
# std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats + asia_feats
df_odd.loc[:, to_scale] = std_scaler.transform(df_odd.loc[:, to_scale])
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,-0.581139,-0.797506,0.067677,-0.568650,0.989373,-1.104495,0.063057,-0.692567,-1,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
1,-0.150950,-0.985462,-0.504153,0.336557,-0.294210,0.243320,-0.082727,-0.242100,0,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
2,-0.667956,-0.463362,0.220165,-0.568650,-1.577794,1.591135,0.195352,-0.373032,-1,1,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
3,-0.522148,-0.713970,-0.161055,-0.568650,0.732656,-0.834932,0.206546,-0.022821,-1,1,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
4,-0.667956,-0.672202,0.372653,-0.568650,-0.550927,0.512883,0.019535,-1.282142,-1,0,...,2,0.866025,5.000000e-01,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37641,0.266439,-0.630434,-0.633768,0.638292,-0.037494,-0.026243,0.181349,1.522708,1,1,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,1,0,2,6.0,7.0
37642,0.739479,-0.379826,-0.725260,0.940028,0.270566,-0.349719,-0.834565,0.598636,1,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,2.0,3.0
37643,-0.011821,-0.672202,-0.505678,0.336557,0.373253,-0.457544,1.456389,2.131324,0,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,0,0,0,7.0,8.0
37644,-0.751990,0.789678,0.677629,-1.172120,-0.448240,0.405058,2.062301,0.368986,-1,0,...,9,-1.000000,-1.836970e-16,-0.972118,-0.234491,2,2,0,10.0,9.0


In [76]:
# 提取预测数据
testing_row_number = df_noNa_new.shape[0]
df_testing_data = df_odd.iloc[-testing_row_number:].loc[:, df_odd.columns != 'Result']
df_testing_data

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
37625,2.631642,0.037854,-0.853350,1.543499,0.989373,-1.104495,-1.157533,1.184803,1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,0,0,0,3.0,8.0
37626,0.572524,-0.630434,-0.679514,0.940028,-1.167047,1.159834,-0.917200,0.153158,1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,1,1,1,5.0,4.0
37627,-0.568339,-0.463362,0.220165,-0.568650,0.886686,-0.996670,2.431444,1.252970,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,2,0,0,8.0,6.0
37628,-0.484862,-0.296290,-0.145806,-0.266914,-0.448240,0.405058,0.590386,0.806744,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,2,0,2,5.0,5.0
37629,-0.290080,-0.881042,-0.237299,0.034821,0.167880,-0.241893,0.641454,0.816659,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,2,5,1,8.0,8.0
37630,-0.011821,-0.296290,-0.572772,0.638292,-1.372420,1.375484,-1.147765,0.161375,1,1,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,1,0,0,3.0,5.0
37631,-0.718599,0.229987,0.677629,-0.870385,-1.064360,1.052009,0.856865,-0.757563,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,2,2,0,9.0,2.0
37632,0.071657,-0.797506,-0.527026,0.336557,1.400120,-1.535796,-0.053148,0.791445,0,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,0,0,0,5.0,5.0
37633,-0.317906,-0.463362,-0.328792,0.034821,-0.037494,-0.026243,-1.306560,-1.648306,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,2,0,0,3.0,0.0
37634,-0.585035,-0.212754,0.143921,-0.568650,-0.037494,-0.026243,-0.139294,-1.067287,-1,0,...,9,-1.0,-1.836970e-16,-0.972118,-0.234491,1,1,0,7.0,3.0


In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.init as init

# 定义 MLP 模型（与之前类似）
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_layers=[256, 128, 32], output_dim=3, dropout_prob=0.3):
        super(MLPModel, self).__init__()

        layers = []
        in_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(in_dim, hidden_dim))
            # self.initialize_weights(layers[-1], method='kaiming')  # 使用Kaiming初始化
            # layers.append(nn.BatchNorm1d(hidden_dim))  # 添加 BatchNorm 层
            layers.append(nn.ELU())
            layers.append(nn.Dropout(dropout_prob))  # 添加 Dropout 层
            in_dim = hidden_dim
            
        layers.append(nn.Linear(in_dim, output_dim))  # 输出层
        # self.initialize_weights(layers[-2], method='kaiming')  # 输出层初始化
        layers.append(nn.Softmax(dim=1))  # 归一化为概率分布
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)  # 将 softmax 归一化到 (0,2) 范围
    
    def initialize_weights(self, layer, method='kaiming'):
        """
        初始化网络的权重
        :param layer: 当前层
        :param method: 初始化方法 ('kaiming' 或 'xavier')
        """
        if isinstance(layer, nn.Linear):
            if method == 'kaiming':
                init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')  # for ELU/Relu
            elif method == 'xavier':
                init.xavier_normal_(layer.weight)  # for tanh or sigmoid activations
            if layer.bias is not None:
                init.zeros_(layer.bias)  # 初始化偏置为0
        elif isinstance(layer, nn.Conv1d):
            if method == 'kaiming':
                init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')
            elif method == 'xavier':
                init.xavier_normal_(layer.weight)
            if layer.bias is not None:
                init.zeros_(layer.bias)  # Conv层的偏置初始化为0
                
# 计算 top-k 准确率
def compute_top_k_accuracy(model, x_test, y_test, k=2):
    model.eval()
    with torch.no_grad():
        outputs = model(x_test)
        _, top_k_indices = torch.topk(outputs, k, dim=1, largest=True, sorted=True)
        correct_top_1 = torch.sum(top_k_indices[:, 0] == y_test).item()
        correct_top_2 = torch.sum((top_k_indices[:, 0] == y_test) | (top_k_indices[:, 1] == y_test)).item()
        top_1_accuracy = correct_top_1 / len(y_test)
        top_2_accuracy = correct_top_2 / len(y_test)
    return top_1_accuracy, top_2_accuracy

In [78]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# eval_model = torch.load("./MLPModel_2025_02_14_18_09_49.pt").to(device)
eval_model = torch.load("./MLPModel_2025_03_11_09_24_28.pt").to(device)
print(eval_model)
# 评估模型的 top-1 和 top-2 准确率
x_val = torch.tensor(df_testing_data.values.astype('float32'), dtype=torch.float32).to(device)
eval_model(x_val)

MLPModel(
  (network): Sequential(
    (0): Linear(in_features=439, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.6, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.6, inplace=False)
    (6): Linear(in_features=64, out_features=16, bias=True)
    (7): ELU(alpha=1.0)
    (8): Dropout(p=0.6, inplace=False)
    (9): Linear(in_features=16, out_features=3, bias=True)
    (10): Softmax(dim=1)
  )
)


tensor([[0.6920, 0.2052, 0.1028],
        [0.5084, 0.2774, 0.2142],
        [0.1735, 0.2418, 0.5847],
        [0.2221, 0.2688, 0.5091],
        [0.2919, 0.2789, 0.4292],
        [0.3989, 0.2900, 0.3112],
        [0.1297, 0.2225, 0.6478],
        [0.3887, 0.2896, 0.3217],
        [0.2649, 0.2944, 0.4408],
        [0.1799, 0.2546, 0.5655],
        [0.3214, 0.2877, 0.3908],
        [0.4448, 0.2692, 0.2860],
        [0.2772, 0.2958, 0.4269],
        [0.4396, 0.2833, 0.2771],
        [0.2855, 0.2888, 0.4257],
        [0.2498, 0.2860, 0.4642],
        [0.4883, 0.2738, 0.2379],
        [0.5279, 0.2699, 0.2022],
        [0.3949, 0.2808, 0.3243],
        [0.1293, 0.2079, 0.6628],
        [0.4183, 0.2742, 0.3075]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# 方法 1：直接创建
numbers = [0,2,0,2,0,0,1,2,1,0,1,2,2,0,1,2,1,1,1,0,1,]
y_val_fix = torch.tensor(numbers).to(device)
top_1_accuracy, top_2_accuracy = compute_top_k_accuracy(eval_model, x_val, y_val_fix, k=2)
print(f"Top-1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top-2 Accuracy: {top_2_accuracy:.4f}")

### 加载模型
# model_name = 'sklearn_mlp_' + local_time + '.joblib'
# model_name = './' + 'sklearn_svm_2025_01_27_16_48_56.joblib'
# model_name = './' + 'sklearn_svm_randsearch_2025_01_27_17_09_20.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_28_21_56_17.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_29_15_12_48.joblib'
model_name = './' + 'sklearn_mlp_2025_01_31_10_13_42.joblib'
mlp = load(model_name)
print(mlp)
predict_val = mlp.predict(df_testing_data)
series_pre = pd.Series(predict_val, name='Predicted')
series_pre
# compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
# compare_result